In [ ]:
import os, json, random

QnA_dir_path = '17000_validated.json'
with open(QnA_dir_path, 'r') as f:
    file_data = json.load(f)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
os.environ['HUGGINGFACE_HUB_CACHE'] = "/data/llm_models/huggingface/hub"

In [ ]:
# defect = []
# def shuffle(data):
#     try:
#         corr = data['options'][int(re.search(r'\d+', data['answer']).group()) - 1].split(': ', 1)[1]
#     except Exception:
#         defect.append(data)
#         return
# 
#     # Shuffle the options
#     random.shuffle(data['options'])
#     
#     # Reassign option numbers based on new order
#     data['options'] = [f'option {i + 1}: {opt.split(": ", 1)[1]}' for i, opt in
#                        enumerate(data['options'])]
#     
#     # Update the answer to reflect the new position of the correct option
#     for i, option in enumerate(data['options'], start=1):
#         if corr in option:
#             data['answer'] = f'option {i}'
#             break

# for jib in file_data:
#     shuffle(jib)
# len(defect)

In [ ]:
file_data = file_data[:1000]

In [ ]:
file_data

In [ ]:
model_name = '/data/llm_models/internlm2-7b-0410-ssl+sft'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map = "auto").eval()

In [ ]:
prompt = """Please answer the following multiple-choice question.
Select only ONE correct option by its number. Start your response with 'The correct option is' followed by the option number ONLY. eg: "The correct option is Option X."
Do not include ANY justification, additional information, restatements of the options, or list multiple options as correct."""

In [ ]:
file_data

In [ ]:
import re
answer_list = [f['answer'] for f in file_data]
answer_list = [re.search(r'\d+', a).group() for a in answer_list]

In [ ]:
# answer_list = [q['completion'] for q in ques]

In [ ]:
question = []
for f in file_data:
    options = ''
    for o in f['options']:
        options += o + '\n'
    sb = prompt + '\n question: ' + f['question'] + '\n options: ' + options + 'The correct option number is:'
    question.append(sb)

In [ ]:
inputs = []
# for q in ques:    
for q in question:
    a = tokenizer(q, return_tensors="pt", padding=True)
    input_ids = a.input_ids.to('cuda')
    inputs.append(input_ids)
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
len(inputs)

In [ ]:
from tqdm import tqdm
output_list = []
for q in tqdm(inputs[:]):
    output_list.append(model.generate(q, max_new_tokens=10))

In [ ]:
lst = [tokenizer.decode(i[0], skip_special_tokens=True) for i in output_list]

In [ ]:
output_list = lst

In [ ]:
after = []
for i, j in zip(output_list, question):
# for i, j in zip(output_list, question):
    after.append(i.replace(j, ''))
    # after.append(i.replace(j, ''))
    

In [ ]:
import re
organized_output_list = []
for i in after:
    temp = re.search(r'\d+', i)
    if temp:
        organized_output_list.append(temp.group())
    else:
        organized_output_list.append('x')
#     

In [ ]:
# import re
# organized_output_list = []
# for i in after:  # Assuming 'after' is a list of strings you're iterating through
#     if ('A) no' or 'B) yes') in i:
#         organized_output_list.append('x')
#         continue
# 
#     if 'Answer' in i:
#         index = i.find('Answer') + len('Answer')
#         i = i[index:].strip()
#     # if 'The' in i:
#     #     index = i.find('The') + len('The')
#     #     i = i[index:].strip()
#     temp = re.search(r'[A-Z]', i)  # This regex matches the first uppercase letter
#     if temp:
#         organized_output_list.append(temp.group())  # Append the first uppercase letter found
#     else:
#         organized_output_list.append('x')  # Append 'x' if no uppercase letter is found


In [ ]:
num_correct = 0
for i in range(len(organized_output_list)):
    if organized_output_list[i] == answer_list[i]:
        num_correct += 1
    # else:
    #     print(file_data[i]['answer'], after[i])

In [ ]:
num_correct / 1000